# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9fe4f8dfa0>
├── 'a' --> tensor([[-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424]])
└── 'x' --> <FastTreeValue 0x7f9fe4f8df70>
    └── 'c' --> tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                        [-0.1458,  0.1677,  0.5882,  0.9782],
                        [ 1.5681,  0.5234,  1.4312, -0.3264]])

In [4]:
t.a

tensor([[-0.4399,  1.0216, -0.5115],
        [ 0.4207,  1.9279, -1.0424]])

In [5]:
%timeit t.a

63.2 ns ± 0.0592 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9fe4f8dfa0>
├── 'a' --> tensor([[-0.4353, -1.2596, -1.1303],
│                   [-0.2871,  0.6893,  0.4100]])
└── 'x' --> <FastTreeValue 0x7f9fe4f8df70>
    └── 'c' --> tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                        [-0.1458,  0.1677,  0.5882,  0.9782],
                        [ 1.5681,  0.5234,  1.4312, -0.3264]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.0589 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424]]),
    x: Batch(
           c: tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                      [-0.1458,  0.1677,  0.5882,  0.9782],
                      [ 1.5681,  0.5234,  1.4312, -0.3264]]),
       ),
)

In [10]:
b.a

tensor([[-0.4399,  1.0216, -0.5115],
        [ 0.4207,  1.9279, -1.0424]])

In [11]:
%timeit b.a

61.3 ns ± 0.0205 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5830,  1.6165,  0.7186],
               [-1.4076, -1.5870, -1.7547]]),
    x: Batch(
           c: tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                      [-0.1458,  0.1677,  0.5882,  0.9782],
                      [ 1.5681,  0.5234,  1.4312, -0.3264]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.0675 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

821 ns ± 1.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 9.52 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9f45ed1610>
├── 'a' --> tensor([[[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]],
│           
│                   [[-0.4399,  1.0216, -0.5115],
│                    [ 0.4207,  1.9279, -1.0424]]])
└── 'x' --> <FastTreeValue 0x7f9f45ed17c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 84.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9f508a8a60>
├── 'a' --> tensor([[-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424],
│                   [-0.4399,  1.0216, -0.5115],
│                   [ 0.4207,  1.9279, -1.0424]])
└── 'x' --> <FastTreeValue 0x7f9f3f27df40>
    └── 'c' --> tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                        [-0.1458,  0.1677,  0.5882,  0.9782],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 67.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.4 µs ± 68.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]],
       
               [[-0.4399,  1.0216, -0.5115],
                [ 0.4207,  1.9279, -1.0424]]]),
    x: Batch(
           c: tensor([[[-0.2820, -0.7833,  1.8021, -1.5923],
                       [-0.1458,  0.1677,  0.5882,  0.9782],
                       [ 1.5681,  0.5234,  1.4312, -0.3264]],
         

In [26]:
%timeit Batch.stack(batches)

76.8 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424],
               [-0.4399,  1.0216, -0.5115],
               [ 0.4207,  1.9279, -1.0424]]),
    x: Batch(
           c: tensor([[-0.2820, -0.7833,  1.8021, -1.5923],
                      [-0.1458,  0.1677,  0.5882,  0.9782],
                      [ 1.5681,  0.5234,  1.4312, -0.3264],
                      [-0.2820, -0.7833,  1.8021, -1.5923],
                      [-0.1458,  

In [28]:
%timeit Batch.cat(batches)

141 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
